In [7]:
!pip install pybacktestchain
!pip install web3


In [8]:
from pybacktestchain.data_module import FirstTwoMoments
from pybacktestchain.broker import Backtest, StopLoss
from pybacktestchain.blockchain import load_blockchain
from datetime import datetime

# Set verbosity for logging
verbose = False  # Set to True to enable logging, or False to suppress it

backtest = Backtest(
    initial_date=datetime(2019, 1, 1),
    final_date=datetime(2020, 1, 1),
    information_class=FirstTwoMoments,
    risk_model=StopLoss,
    name_blockchain='backtest',
    verbose=verbose
)

backtest.run_backtest()

block_chain = load_blockchain('backtest')
print(str(block_chain))
# check if the blockchain is valid
print(block_chain.is_valid())

INFO:root:Running backtest from 2019-01-01 00:00:00 to 2020-01-01 00:00:00.
INFO:root:Retrieving price data for universe
c:\Users\ASUS\anaconda3\Lib\site-packages\pandas\core\frame.py:11211: RuntimeWarning: Degrees of freedom <= 0 for slice
  base_cov = np.cov(mat.T, ddof=ddof)
c:\Users\ASUS\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
c:\Users\ASUS\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
INFO:root:Stop loss triggered for NVDA at 2019-01-03 00:00:00. Selling all shares.
INFO:root:Stop loss triggered for NFLX at 2019-01-03 00:00:00. Selling all shares.
INFO:root:Stop loss triggered for CSCO at 2019-01-03 00:00:00. Selling all shares.
INFO:root:-----------------------------------
INFO:root:Rebalancing portfolio at 2019-01-31 00:00:00
INFO:root:-----------------------------------
INFO:r

--------------------------------------------------------------------------------
Block 0
--------------------------------------------------------------------------------
Backtest: Genesis Block
Timestamp: 1735640890.7700677
Hash: 5e687161c257fea2c315496aee5091b8aacc76e9d560f933a574a6625b31a713
Previous Hash: 0
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Block 1
--------------------------------------------------------------------------------
Backtest: AmberWolfBlacksmith
Timestamp: 1735640902.6022513
Hash: 238dd8c6591c8af3b5708ef655fb38853213ba7293fdd440e9a627fcef88d5d3
Previous Hash: 5e687161c257fea2c315496aee5091b8aacc76e9d560f933a574a6625b31a713
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Block 2
--------------------------------------------------------

First, we create a code that leverages pybacktestchain to run a backtest on a user-defined trading strategy, evaluating its profitability and innovativeness. 
It calculates key performance metrics like net profit, Sharpe ratio, and maximum drawdown, while comparing the strategy to historical ones using pattern correlation. 
Results are stored in a blockchain to ensure transparency and traceability.

In [11]:
import pybacktestchain.blockchain as blockchain
print(dir(blockchain))

['Block', 'Blockchain', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'dataclass', 'field', 'hashlib', 'load_blockchain', 'os', 'pickle', 'remove_blockchain', 'time']


In [49]:
from pybacktestchain.data_module import FirstTwoMoments
from pybacktestchain.broker import Backtest, StopLoss
from pybacktestchain.blockchain import load_blockchain
from datetime import datetime
import pandas as pd
import numpy as np
from io import StringIO
import pickle

# Ensures that the user's strategy is correctly applied to the market data.
def apply_user_strategy(strategy_function, data):
    """
    Apply the user's strategy to the data.
    :param strategy_function: User-defined strategy function.
    :param data: Historical market data (DataFrame).
    :return: DataFrame with added 'Signal' column.
    """
    try:
        # Check if the strategy function can be applied
        data = strategy_function(data)
        if 'Signal' not in data.columns:
            raise ValueError("The strategy function must add a 'Signal' column to the data.")
        return data
    except Exception as e:
        print(f"Error applying user strategy: {e}")
        raise

# Initialize the backtest
backtest = Backtest(
    initial_date=datetime(2019, 1, 1),
    final_date=datetime(2020, 1, 1),
    information_class=FirstTwoMoments,
    risk_model=StopLoss,
    name_blockchain='user_strategy_analysis',
    verbose=True
)

# Example of user-defined strategy (replace with user input)
def user_defined_strategy(data):
    """
    Example: Moving Average Crossover Strategy.
    Users can replace this with their own strategy.
    """
    short_ma = data['Close'].rolling(window=10).mean()
    long_ma = data['Close'].rolling(window=50).mean()
    data['Signal'] = np.where(short_ma > long_ma, 1, -1)  # 1: Buy, -1: Sell
    return data

# Apply the user's strategy to the backtest
try:
    backtest.strategy = lambda data: apply_user_strategy(user_defined_strategy, data)
    backtest.run_backtest()
except Exception as e:
    print(f"Failed to run the backtest: {e}")

# Load the blockchain
blockchain = load_blockchain('user_strategy_analysis')

# Extract trades from a single block
def extract_trades_from_block(block):
    """
    Extract trades from a block's data field.
    """
    if block.data:
        try:
            # Parse the data string into a DataFrame
            df = pd.read_csv(StringIO(block.data), delim_whitespace=True)
            return df
        except Exception as e:
            print(f"Error parsing block data: {e}")
            return None
    return None

# Extract all trades from the blockchain
def extract_all_trades(blockchain):
    """
    Extract trades from all blocks in the blockchain.
    """
    trades = []
    for block in blockchain.chain:
        trades_df = extract_trades_from_block(block)
        if trades_df is not None:
            trades.append(trades_df)
    # Combine all trade data into a single DataFrame
    if trades:
        return pd.concat(trades, ignore_index=True)
    return pd.DataFrame()

# Analyze profitability
def calculate_profitability(trades_df):
    """
    Calculate net profit and other key performance metrics.
    """
    if trades_df.empty:
        print("No trades found.")
        return {"Net Profit": 0, "Sharpe Ratio": 0}

    # Calculate net profit
    trades_df['Profit'] = trades_df['Quantity'] * trades_df['Price']
    net_profit = trades_df['Profit'].sum()

    # Example: Calculate Sharpe Ratio
    sharpe_ratio = trades_df['Profit'].mean() / trades_df['Profit'].std() if trades_df['Profit'].std() > 0 else 0

    print(f"Net Profit: {net_profit}")
    print(f"Sharpe Ratio: {sharpe_ratio}")
    return {"Net Profit": net_profit, "Sharpe Ratio": sharpe_ratio}

# Analyze innovation of the strategy
def analyze_innovation(trades_df):
    """
    Analyze the innovation of the user's strategy.
    Compare the strategy's signals or trading patterns against historical strategies.
    """
    if trades_df.empty:
        print("No trades found to analyze innovation.")
        return {"Innovation Score": 0}

    # Example: Compare the user's trades to a dummy historical pattern
    # (In a real-world scenario, you would load historical strategies from a database)
    historical_signals = np.random.choice([-1, 1], size=len(trades_df))  # Dummy historical signals
    user_signals = trades_df['Signal'] if 'Signal' in trades_df else []

    # Calculate innovation as 1 - correlation with historical signals
    correlation = np.corrcoef(user_signals, historical_signals)[0, 1] if len(user_signals) > 1 else 0
    innovation_score = 1 - correlation

    print(f"Innovation Score: {innovation_score}")
    return {"Innovation Score": innovation_score}

# Save blockchain to a file using pickle
def save_blockchain(blockchain, filename='blockchain.pkl'):
    """Save the blockchain object to a file."""
    with open(filename, 'wb') as file:
        pickle.dump(blockchain, file)
    print(f"Blockchain saved to {filename}")

# Extract and analyze trades
trades_df = extract_all_trades(blockchain)
print(trades_df.head())  # Display the first few trades

# Calculate profitability metrics
profitability_metrics = calculate_profitability(trades_df)
print(profitability_metrics)

# Analyze innovation
innovation_metrics = analyze_innovation(trades_df)
print(innovation_metrics)

# Save the blockchain
save_blockchain(blockchain)


INFO:root:Running backtest from 2019-01-01 00:00:00 to 2020-01-01 00:00:00.
INFO:root:Retrieving price data for universe
c:\Users\ASUS\anaconda3\Lib\site-packages\pandas\core\frame.py:11211: RuntimeWarning: Degrees of freedom <= 0 for slice
  base_cov = np.cov(mat.T, ddof=ddof)
c:\Users\ASUS\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
c:\Users\ASUS\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
INFO:root:Stop loss triggered for NVDA at 2019-01-03 00:00:00. Selling all shares.
INFO:root:Stop loss triggered for NFLX at 2019-01-03 00:00:00. Selling all shares.
INFO:root:Stop loss triggered for CSCO at 2019-01-03 00:00:00. Selling all shares.
INFO:root:-----------------------------------
INFO:root:Rebalancing portfolio at 2019-01-31 00:00:00
INFO:root:-----------------------------------
INFO:r

         Date Action Ticker  Quantity      Price          Cash
0  2019-01-31    BUY   NVDA    290529   3.407105  10137.200000
1  2019-01-31    BUY   TSLA       492  20.584667      9.544189
2  2019-02-28   SELL   NVDA     12749   3.853979  49143.920000
3  2019-02-28    BUY   TSLA      2342  20.982668      2.515596
4  2019-03-06   SELL   TSLA      2834  18.436001  52250.140000
Net Profit: 1354852705.144972
Sharpe Ratio: 0.491131304237656
{'Net Profit': 1354852705.144972, 'Sharpe Ratio': 0.491131304237656}
Innovation Score: 1
{'Innovation Score': 1}
Blockchain saved to blockchain.pkl


In [44]:
from pybacktestchain.data_module import FirstTwoMoments
from pybacktestchain.broker import Backtest, StopLoss
from pybacktestchain.blockchain import load_blockchain
from datetime import datetime
import pandas as pd
import numpy as np
from io import StringIO
import pickle

# Ensures that the user's strategy is correctly applied to the market data.
def apply_user_strategy(strategy_function, data):
    try:
        data = strategy_function(data)
        if 'Signal' not in data.columns:
            raise ValueError("The strategy function must add a 'Signal' column to the data.")
        return data
    except Exception as e:
        print(f"Error applying user strategy: {e}")
        raise

# Define historical strategies
def create_historical_strategies():
    """
    Create a list of basic historical strategies.
    Each strategy function operates on the market data and generates signals.
    :return: List of strategy functions.
    """
    strategies = []

    # Strategy 1: Always buy
    def strategy_buy(data):
        data['Signal'] = 1  # Always buy
        return data
    strategies.append(strategy_buy)

    # Strategy 2: Always sell
    def strategy_sell(data):
        data['Signal'] = -1  # Always sell
        return data
    strategies.append(strategy_sell)

    # Strategy 3: Random signals
    def strategy_random(data):
        data['Signal'] = np.random.choice([-1, 1], size=len(data))
        return data
    strategies.append(strategy_random)

    # Strategy 4: Short-term momentum
    def strategy_short_momentum(data):
        short_term_diff = data['Close'].diff()
        data['Signal'] = np.where(short_term_diff > 0, 1, -1)
        return data
    strategies.append(strategy_short_momentum)

    # Strategy 5: Long-term momentum
    def strategy_long_momentum(data):
        long_term_mean = data['Close'].rolling(window=20).mean().diff()
        data['Signal'] = np.where(long_term_mean > 0, 1, -1)
        return data
    strategies.append(strategy_long_momentum)

    # Strategy 6: Mean reversion
    def strategy_mean_reversion(data):
        rolling_mean = data['Close'].rolling(window=10).mean()
        data['Signal'] = np.where(data['Close'] < rolling_mean, 1, -1)
        return data
    strategies.append(strategy_mean_reversion)

    # Strategy 7: Moving average crossover
    def strategy_moving_average(data):
        short_ma = data['Close'].rolling(window=5).mean()
        long_ma = data['Close'].rolling(window=15).mean()
        data['Signal'] = np.where(short_ma > long_ma, 1, -1)
        return data
    strategies.append(strategy_moving_average)

    # Strategy 8: Price above moving average
    def strategy_above_ma(data):
        rolling_mean = data['Close'].rolling(window=10).mean()
        data['Signal'] = np.where(data['Close'] > rolling_mean, 1, -1)
        return data
    strategies.append(strategy_above_ma)

    # Strategy 9: Volatility breakout
    def strategy_volatility_breakout(data):
        rolling_mean = data['Close'].rolling(window=10).mean()
        rolling_std = data['Close'].rolling(window=10).std()
        data['Signal'] = np.where(data['Close'] > rolling_mean + 2 * rolling_std, 1, -1)
        return data
    strategies.append(strategy_volatility_breakout)

    # Strategy 10: RSI-based strategy
    def strategy_rsi(data):
        delta = data['Close'].diff()
        gain = np.where(delta > 0, delta, 0)
        loss = np.where(delta < 0, -delta, 0)
        avg_gain = pd.Series(gain).rolling(window=14).mean()
        avg_loss = pd.Series(loss).rolling(window=14).mean()
        rs = avg_gain / avg_loss
        rsi = 100 - (100 / (1 + rs))
        data['Signal'] = np.where(rsi < 30, 1, np.where(rsi > 70, -1, 0))
        return data
    strategies.append(strategy_rsi)

    return strategies

historical_strategies = create_historical_strategies()

# Analyze innovation of the strategy
def analyze_innovation(trades_df, historical_strategies):
    if trades_df.empty:
        print("No trades found to analyze innovation.")
        return {"Innovation Score": 0}

    user_signals = trades_df['Signal'] if 'Signal' in trades_df.columns else []

    historical_signals_list = []
    for strategy in historical_strategies:
        try:
            historical_data = trades_df.copy()
            historical_data = strategy(historical_data)
            if 'Signal' in historical_data.columns:
                historical_signals_list.append(historical_data['Signal'].to_numpy())
        except Exception as e:
            print(f"Error applying historical strategy: {e}")

    correlations = []
    for historical_signals in historical_signals_list:
        if len(user_signals) == len(historical_signals):
            correlation = np.corrcoef(user_signals, historical_signals)[0, 1]
            correlations.append(correlation)

    avg_correlation = np.mean(correlations) if correlations else 0
    innovation_score = 1 - avg_correlation

    print(f"Innovation Score: {innovation_score}")
    return {"Innovation Score": innovation_score}

# Initialize the backtest
backtest = Backtest(
    initial_date=datetime(2019, 1, 1),
    final_date=datetime(2020, 1, 1),
    information_class=FirstTwoMoments,
    risk_model=StopLoss,
    name_blockchain='user_strategy_analysis',
    verbose=True
)

# Example of user-defined strategy
def user_defined_strategy(data):
    short_ma = data['Close'].rolling(window=10).mean()
    long_ma = data['Close'].rolling(window=50).mean()
    data['Signal'] = np.where(short_ma > long_ma, 1, -1)
    return data

try:
    backtest.strategy = lambda data: apply_user_strategy(user_defined_strategy, data)
    backtest.run_backtest()
except Exception as e:
    print(f"Failed to run the backtest: {e}")

blockchain = load_blockchain('user_strategy_analysis')

# Extract trades and analyze innovation
def extract_all_trades(blockchain):
    trades = []
    for block in blockchain.chain:
        if block.data:
            try:
                trades_df = pd.read_csv(StringIO(block.data), delim_whitespace=True)
                trades.append(trades_df)
            except Exception as e:
                print(f"Error parsing block data: {e}")
    return pd.concat(trades, ignore_index=True) if trades else pd.DataFrame()

trades_df = extract_all_trades(blockchain)

# Placeholder for calculate_profitability function
def calculate_profitability(trades_df):
    if trades_df.empty:
        print("No trades found.")
        return {"Net Profit": 0, "Sharpe Ratio": 0}

    trades_df['Profit'] = trades_df['Quantity'] * trades_df['Price']
    net_profit = trades_df['Profit'].sum()
    sharpe_ratio = trades_df['Profit'].mean() / trades_df['Profit'].std() if trades_df['Profit'].std() > 0 else 0

    print(f"Net Profit: {net_profit}")
    print(f"Sharpe Ratio: {sharpe_ratio}")
    return {"Net Profit": net_profit, "Sharpe Ratio": sharpe_ratio}

profitability_metrics = calculate_profitability(trades_df)
print(profitability_metrics)

innovation_metrics = analyze_innovation(trades_df, historical_strategies)
print(innovation_metrics)

# Save blockchain
def save_blockchain(blockchain, filename='blockchain.pkl'):
    with open(filename, 'wb') as file:
        pickle.dump(blockchain, file)
    print(f"Blockchain saved to {filename}")

save_blockchain(blockchain)


INFO:root:Running backtest from 2019-01-01 00:00:00 to 2020-01-01 00:00:00.
INFO:root:Retrieving price data for universe


[<function create_historical_strategies.<locals>.strategy_buy at 0x000001DC1504DC60>, <function create_historical_strategies.<locals>.strategy_sell at 0x000001DC15E7ECA0>, <function create_historical_strategies.<locals>.strategy_random at 0x000001DC15E7CD60>, <function create_historical_strategies.<locals>.strategy_short_momentum at 0x000001DC15E7EB60>, <function create_historical_strategies.<locals>.strategy_long_momentum at 0x000001DC15E7DD00>, <function create_historical_strategies.<locals>.strategy_mean_reversion at 0x000001DC15E7F420>, <function create_historical_strategies.<locals>.strategy_moving_average at 0x000001DC15E7D940>, <function create_historical_strategies.<locals>.strategy_above_ma at 0x000001DC15E7E520>, <function create_historical_strategies.<locals>.strategy_volatility_breakout at 0x000001DC15E7CB80>, <function create_historical_strategies.<locals>.strategy_rsi at 0x000001DC15E7E700>]


c:\Users\ASUS\anaconda3\Lib\site-packages\pandas\core\frame.py:11211: RuntimeWarning: Degrees of freedom <= 0 for slice
  base_cov = np.cov(mat.T, ddof=ddof)
c:\Users\ASUS\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
c:\Users\ASUS\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
INFO:root:Stop loss triggered for NVDA at 2019-01-03 00:00:00. Selling all shares.
INFO:root:Stop loss triggered for NFLX at 2019-01-03 00:00:00. Selling all shares.
INFO:root:Stop loss triggered for CSCO at 2019-01-03 00:00:00. Selling all shares.
INFO:root:-----------------------------------
INFO:root:Rebalancing portfolio at 2019-01-31 00:00:00
INFO:root:-----------------------------------
INFO:root:Rebalancing portfolio at 2019-02-28 00:00:00
INFO:root:Stop loss triggered for TSLA at 2019-03-06 00:00:00. Selling a

Net Profit: 1143106718.414781
Sharpe Ratio: 0.491270304765847
{'Net Profit': 1143106718.414781, 'Sharpe Ratio': 0.491270304765847}
Error applying historical strategy: 'Close'
Error applying historical strategy: 'Close'
Error applying historical strategy: 'Close'
Error applying historical strategy: 'Close'
Error applying historical strategy: 'Close'
Error applying historical strategy: 'Close'
Error applying historical strategy: 'Close'
Innovation Score: 1
{'Innovation Score': 1}
Blockchain saved to blockchain.pkl


In [50]:
!pip install eth-brownie web3

  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl.metadata (9.7 kB)
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 1.6/1.6 MB 10.8 MB/s eta 0:00:00
Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 1.4/1.4 MB 14.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   -----------------------------

  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [5 lines of output]
      running bdist_wheel
      running build
      running build_ext
      building 'lru' extension
      error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for lru-dict
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (lru-dict)
'brownie' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


In [51]:
!brownie init

'brownie' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


In [3]:
import hashlib
import time
from dataclasses import dataclass, field

@dataclass
class Block:
    data: str
    previous_hash: str = ''
    timestamp: float = field(default_factory=time.time)
    hash: str = field(init=False)
    rewards: int = 0  # Reward for the block

    def __post_init__(self):
        self.hash = self.calculate_hash

    @property
    def calculate_hash(self):
        return hashlib.sha256(
            (str(self.timestamp) + self.data + self.previous_hash).encode()
        ).hexdigest()


@dataclass
class Blockchain:
    chain: list = field(default_factory=list)

    def __post_init__(self):
        self.chain.append(self.create_genesis_block())

    def create_genesis_block(self):
        return Block('Genesis Block', '0')

    def add_block(self, data: str, rewards: int):
        previous_block = self.chain[-1]
        new_block = Block(data, previous_block.hash, rewards=rewards)
        self.chain.append(new_block)

    def is_chain_valid(self):
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            previous_block = self.chain[i - 1]
            if current_block.previous_hash != previous_block.hash:
                return False
        return True


In [4]:
pragma solidity ^0.8.0;

contract RewardToken {
    string public name = "Reward Token";
    string public symbol = "RWD";
    uint8 public decimals = 18;
    uint256 public totalSupply;

    mapping(address => uint256) public balanceOf;

    event Transfer(address indexed from, address indexed to, uint256 value);

    constructor(uint256 initialSupply) {
        totalSupply = initialSupply * 10 ** uint256(decimals);
        balanceOf[msg.sender] = totalSupply; // Assign all tokens to the creator
    }

    function transfer(address to, uint256 value) public returns (bool success) {
        require(balanceOf[msg.sender] >= value, "Insufficient balance");
        balanceOf[msg.sender] -= value;
        balanceOf[to] += value;
        emit Transfer(msg.sender, to, value);
        return true;
    }
}


SyntaxError: invalid syntax (72485702.py, line 1)